In [14]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import glob
import re
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Tuple, List, Callable, Any
import torch.nn as nn
from torch import Tensor
from torch.optim.lr_scheduler import StepLR

$$output\_shape = \frac{input\_shape + 2{\times}padding - dilation{\times}(kernel\_size - 1) - 1}{stride} + 1$$

In [2]:
class path_1(Dataset):

    def __init__(self, root_path):
        self.mDataX = []
        self.mDataY = []

        for img_path in glob.glob(root_path + r'/*'):
            img = Image.open(img_path)
            img_data = np.array(img, dtype = float)/255
            #img_data = img_data[np.newaxis,:,:]
            #img_data = img_data.reshape(-1)
            self.mDataX.append(img_data)
            pattern = '[/\\\.]'
            img_na = re.split(pattern, img_path)[-2]
            
            if list(img_na)[-1] == 'T':
                self.mDataY.append([1])
            elif list(img_na)[-1] == 'F':
                self.mDataY.append([0])
    
        self.mDataX = torch.permute(torch.tensor(self.mDataX), (0,3,1,2))
        self.mDataY = torch.tensor(self.mDataY)
        
    def __getitem__(self, data_index):
        input_tensor = torch.tensor(self.mDataX[data_index])
        output_tensor = torch.tensor(self.mDataY[data_index])
        return input_tensor, output_tensor

    def __len__(self):
        return len(self.mDataX)

#train_set = path_1(r'/Users/wz/OneDrive/OneDrive - Kyoto University/Project/Yuyake_recog/pro_set') #Mac
train_set = path_1(r'C:/Users/yaoho/OneDrive - Kyoto University/Project/Yuyake_recog/pro_set') #Win
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 1, shuffle = True, num_workers = 0)  # , num_workers=8

<ipython-input-2-9556de0660c5>:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  self.mDataX = torch.permute(torch.tensor(self.mDataX), (0,3,1,2))


In [8]:
for i, j in train_loader:
    print(i.shape)

torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 329, 40])
torch.Size([1, 3, 32

<ipython-input-7-c9293b050e94>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(self.mDataX[data_index])
<ipython-input-7-c9293b050e94>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output_tensor = torch.tensor(self.mDataY[data_index])


In [4]:
#test_set = path_1(r'/Users/wz/OneDrive/OneDrive - Kyoto University/Project/Yuyake_recog/test_set') #Mac
test_set = path_1(r'C:/Users/yaoho/OneDrive - Kyoto University/Project/Yuyake_recog/test_set') #Win
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 1, shuffle = False)

In [5]:
class BasicConv2d(torch.nn.Module):
    def __init__(self, in_channels: int, out_channels: int, **kwargs: Any) -> None:
        super().__init__()
        self.conv = torch.nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)#bias=False
        self.bn = torch.nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

class Inception(torch.nn.Module):
    def __init__(
        self, 
        in_channels: int,
        ch1x1: int,
        ch3x3red: int,
        ch3x3: int,
        ch5x5red: int,
        ch5x5: int,
        pool_proj: int,
        conv_block: Optional[Callable[..., torch.nn.Module]] = None,
    ) -> None:
        super().__init__()
        if conv_block is None:
            conv_block = BasicConv2d
        self.branch1 = conv_block(in_channels, ch1x1, kernel_size = 1)
        
        self.branch2 = nn.Sequential(
            conv_block(in_channels, ch3x3red, kernel_size=1), 
            conv_block(ch3x3red, ch3x3, kernel_size=3, padding=1)
        )
        
        self.branch3 = nn.Sequential(
            conv_block(in_channels, ch5x5red, kernel_size=1),
            conv_block(ch5x5red, ch5x5, kernel_size=3, padding=1)
        )
        
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1, ceil_mode=True),
            conv_block(in_channels, pool_proj, kernel_size=1)
        )

    def _forward(self, x: Tensor) -> List[Tensor]:
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)

        outputs = [branch1, branch2, branch3, branch4]
        return outputs

    def forward(self, x: Tensor) -> Tensor:
        outputs = self._forward(x)
        return torch.cat(outputs, 1)
    
    
class googleNet(torch.nn.Module):
    def __init__(self,
                dropout = 0.2,
                num_classes = 1):
        super().__init__()
        
        self.conv1 = BasicConv2d(3, 64, kernel_size=5, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(3, stride=2, ceil_mode=True)
        self.conv2 = BasicConv2d(64, 64, kernel_size=1)
        self.conv3 = BasicConv2d(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(3, stride=2, ceil_mode=True)
        
        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(3, stride=2, ceil_mode=True)
        
        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True)
        
        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)
    
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout(p = dropout)
        
        self.fc1 = torch.nn.Linear(1024, 64)
        self.fc2 = torch.nn.Linear(64, 16)
        self.fc3 = torch.nn.Linear(16, num_classes)
        
    def forward(self, x):
        batch_size = x.size(0)
        x = self.maxpool1(self.conv1(x))
        x = self.conv3(self.conv2(x))
        x = self.maxpool2(x)
        
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        x = self.inception4a(x)
        
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        
        x = self.inception4e(x)
        x = self.maxpool4(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        
        x = self.avgpool(x)
        
        x = x.view(batch_size, -1)#flatten
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))

        return x

def train(epoch):
    loss_val = []
    running_loss = 0.
    
    for batch_idx, data in enumerate(train_loader, 0):#minibatch..
        reg = 0.
        inputs, labels = data#scalar X, Y
        inputs = inputs.float()
        labels = labels.float()
        optimizer.zero_grad()#optimizer RESET

        #forward #backward
        outputs = model(inputs)
        #for params in model.parameters():
            #reg += 0.5 * (params ** 2).sum()
        loss = criterion(outputs, labels) + 0.7 * reg
        loss.backward()
        
        optimizer.step()
        running_loss += loss.item()
        loss_val.append(running_loss)
        if batch_idx % len(train_set) == len(train_set) - 1 and epoch % 100 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss))
            running_loss = 0.
        if batch_idx % len(train_set) == len(train_set) - 1 and epoch == 999:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss))
            running_loss = 0.
    return np.mean(loss_val) 

def test():
    correct, total = 0, 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.float()
            labels = labels.float()
            outputs = model(images)
            predicted, _ = torch.max(outputs.data, dim = 1)
            #total += torch.squeeze(labels)
            predicted = torch.squeeze(predicted)
            correct += (torch.round(predicted) == labels).sum().item()
            print(f'predict:{torch.round(predicted)}\nTrue label:{labels}')
    print('Accuracy on test set: %d %%' % (100 * correct / len(test_set)))    

In [ ]:
#Training process
model = googleNet()
criterion = torch.nn.BCELoss(size_average = True)#??##?_?
optimizer = optim.SGD(model.parameters(), lr = 0.00075, momentum = 0.9)
#optimizer = optim.Adam(model.parameters(), lr = 0.0075)
scheduler = StepLR(optimizer, step_size=30, gamma=0.96)

if __name__ == '__main__':
    loss_curv = []
    for epoch in range(1000):
        #train(epoch)
        loss_val = train(epoch)
        loss_curv.append(loss_val)
        scheduler.step()
    plt.plot(loss_curv)
    test()

<ipython-input-2-9556de0660c5>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(self.mDataX[data_index])
<ipython-input-2-9556de0660c5>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output_tensor = torch.tensor(self.mDataY[data_index])


[1,   149] loss: 103.223
[101,   149] loss: 0.046
[201,   149] loss: 0.011


In [7]:
 torch.save(model.state_dict(), 
           'C:/Users/yaoho/OneDrive - Kyoto University/Project/Yuyake_recog/models_cache/googlenet_decay1.pth')

In [10]:
#testing process
PATH = 'C:/Users/yaoho/OneDrive - Kyoto University/Project/Yuyake_recog/models_cache/googlenet_decay1.pth'
model = googleNet()
model.load_state_dict(torch.load(PATH))
model.eval()
test()

predict:0.0
True label:tensor([[0.]])
predict:1.0
True label:tensor([[0.]])
predict:0.0
True label:tensor([[0.]])
predict:1.0
True label:tensor([[1.]])
predict:1.0
True label:tensor([[1.]])
Accuracy on test set: 80 %


<ipython-input-2-9556de0660c5>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(self.mDataX[data_index])
<ipython-input-2-9556de0660c5>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output_tensor = torch.tensor(self.mDataY[data_index])
